In [1]:
import pysam
from pysam import VariantFile as vcf
import operator
from math import log2
import pandas as pd
from pandas import DataFrame as dataframe
import matplotlib.pyplot as plt
import numpy as np
from scipy.spatial.distance import pdist, squareform
import scipy
import  os
import os.path
import matplotlib.colors as mcolors
from scipy import stats

In [4]:
#/data2/wangxuedong/mhc_test_data/first_review/race/split_chr1_xac.csv
def integrate_csv_chromosome(filespath):
    dic_file=dict()
    for i in range(22):
        dic_file["chr"+str(i+1)]=[]
    #print(dic_file)
    for filepath in filespath:
        if filepath.split("/")[-1]=="posmhc.csv":
            dic_file["chr6"].append(filepath)
        else:
            dic_file[filepath.split("/")[-1].split(".")[0].split("_")[1]].append(filepath)
    #print(dic_file)
    for chr_,file_paths in dic_file.items():
        merged_data = pd.DataFrame()
        for file in file_paths:
            data = pd.read_csv(file)
            merged_data = merged_data.append(data, ignore_index=True)
        merged_data_sorted = merged_data.sort_values(by=merged_data.columns[0])
        merged_folderpath="/data2/wangxuedong/mhc_test_data/first_review/race_integration/"+chr_+".csv"
        merged_data_sorted = merged_data_sorted.drop_duplicates(subset=merged_data_sorted.columns[0])
        if chr_=="chr6":
            merged_data_ls=merged_data_sorted[merged_data_sorted[merged_data_sorted.columns[0]] < 29720403]
            merged_data_ls.to_csv("/data2/wangxuedong/mhc_test_data/first_review/race_integration/chr6_partA.csv",index=False)
            merged_data_mid=merged_data_sorted[(merged_data_sorted[merged_data_sorted.columns[0]] >= 29720403) & (merged_data_sorted[merged_data_sorted.columns[0]] <= 33129982)]
            merged_data_mid.to_csv("/data2/wangxuedong/mhc_test_data/first_review/race_integration/chr6_mhcpart.csv",index=False)
            merged_data_bt=merged_data_sorted[merged_data_sorted[merged_data_sorted.columns[0]] > 33129982 ]
            merged_data_bt.to_csv("/data2/wangxuedong/mhc_test_data/first_review/race_integration/chr6_partC.csv",index=False)           
        else:
            merged_data_sorted.to_csv(merged_folderpath, index=False)
      
            
need_merge_filespath="/data2/wangxuedong/mhc_test_data/first_review/race/"
filespath=[need_merge_filespath+file for file in os.listdir(need_merge_filespath)]
# filespath
integrate_csv_chromosome(filespath)
        

/tmp/ipykernel_3138681/1894332975.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  merged_data = merged_data.append(data, ignore_index=True)


/tmp/ipykernel_3138681/1894332975.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  merged_data = merged_data.append(data, ignore_index=True)
/tmp/ipykernel_3138681/1894332975.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  merged_data = merged_data.append(data, ignore_index=True)
/tmp/ipykernel_3138681/1894332975.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  merged_data = merged_data.append(data, ignore_index=True)
/tmp/ipykernel_3138681/1894332975.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  merged_data = merged_data.append(data, ignore_index=True)
/tmp/ipykernel_3138681/1894332975.py:17: FutureWarning: The fram

In [2]:
fileName = '../../1000_population.tsv'


lines = []
categories = []


with open(fileName, 'r') as f:
    text = f.read()

lines = text.split('\n')
for l in lines:
    
    categories.append(l.split('\t')[-1])
allcategories=sorted(list(set(categories)))


In [3]:
#For each race of one csv file, average every 1000 P and put it in the csv file
def getAveragePer1000(filepath,n,filename):
    csv_df= pd.read_csv(filepath, low_memory = False)
    # csv_df = pd.DataFrame(csv_data)
    for col in allcategories:
        averages = [np.mean(csv_df.loc[i:i+n-1, col]) for i in range(0, csv_df.shape[0], n)]
        np.savetxt('/data2/wangxuedong/mhc_test_data/first_review/race_average_bin_1000/'+col+"_"+filename+'.csv', averages, delimiter = ',')


In [4]:
#run every csvfile in the 'race' folder
for curDir, dirs, files in os.walk(top="/data2/wangxuedong/mhc_test_data/first_review/race_integration/"):
    for file in files:
        filename=file.split(".")[0]
        getAveragePer1000(filepath=curDir+file,n=1000,filename=file.split(".")[0])

In [5]:
#for caculating distribution for different race
arrforDistribution_different_race=dict()
for category in allcategories:
    arrforDistribution_different_race[category]=[]

for curDir, dirs, files in os.walk(top="/data2/wangxuedong/mhc_test_data/first_review/race_average_bin_1000/"):
    for file in files:
        for category in allcategories:
            if file.startswith(category):
                csv_data=np.loadtxt(open(curDir+file,"rb"),delimiter=",",skiprows=0)
                arrforDistribution_different_race[category]+=csv_data.tolist()
                
        

In [6]:
mean_dict_different_race=dict()
std_dict_different_race=dict()
for category in allcategories:
    mean_dict_different_race[category]=0
    std_dict_different_race[category]=0

for category in allcategories:
    mean_dict_different_race[category]=np.mean(arrforDistribution_different_race[category])
    std_dict_different_race[category]=np.std(arrforDistribution_different_race[category])


In [7]:
mean_dict_different_race#mean for every race

{'ACB': 0.046621686769679045,
 'ASW': 0.0465892685112964,
 'BEB': 0.035970363185117774,
 'CDX': 0.03266259792789275,
 'CEU': 0.03488434818325704,
 'CHB': 0.03285897897059889,
 'CHS': 0.032796236687928325,
 'CLM': 0.036503986698316906,
 'ESN': 0.04643492431530403,
 'FIN': 0.034935842703769975,
 'GBR': 0.03498406394197642,
 'GIH': 0.035593525252319486,
 'GWD': 0.045836194805001416,
 'IBS': 0.035140910353782764,
 'ITU': 0.0347164844670136,
 'JPT': 0.03273063391657775,
 'KHV': 0.033023382087129426,
 'LWK': 0.0466350005070252,
 'MSL': 0.047257776857587634,
 'MXL': 0.03529323182476227,
 'PEL': 0.03222809053641309,
 'PJL': 0.03450825905794522,
 'PUR': 0.037936602232609126,
 'STU': 0.03454596363552346,
 'TSI': 0.03508890911947475,
 'YRI': 0.04649868546958651}

In [8]:
std_dict_different_race#standard deviation for every race

{'ACB': 0.01660837078934563,
 'ASW': 0.016852430054156438,
 'BEB': 0.016339814570213686,
 'CDX': 0.01657929168371163,
 'CEU': 0.01615285680476383,
 'CHB': 0.0167769938917591,
 'CHS': 0.016361466363591634,
 'CLM': 0.016076712212741314,
 'ESN': 0.016439364809682746,
 'FIN': 0.016395558138330903,
 'GBR': 0.01654186443581865,
 'GIH': 0.016487954511321803,
 'GWD': 0.01637735033664222,
 'IBS': 0.01625740426618882,
 'ITU': 0.015999114837619923,
 'JPT': 0.016777856667394144,
 'KHV': 0.016805860157616894,
 'LWK': 0.01656542896110204,
 'MSL': 0.0168657687428544,
 'MXL': 0.01631757749279766,
 'PEL': 0.015903810806753623,
 'PJL': 0.015785059411911464,
 'PUR': 0.016016827217755162,
 'STU': 0.01615897181252148,
 'TSI': 0.016499539576161064,
 'YRI': 0.01656520483641468}

In [9]:
#calculate p-value, if p-value>0.95 mark it as 1, otherwise, mark it as 0
def compareWith0_01Pvalue(mean_arr,std_arr,filename,filepath):

    arr=[]
    csv_matrix = np.loadtxt(open(filepath,"rb"),delimiter=",",skiprows=0)
    for i in range(len(csv_matrix)):
        mid_res=stats.norm.cdf(csv_matrix[i], mean_arr, std_arr)

        if mid_res>0.95:
            arr.append(1)
        else:
            arr.append(0)
    np.savetxt("../race_compare_with_01pvalue/"+filename+".csv", arr, delimiter = ',')
    return filename

In [10]:
for curDir, dirs, files in os.walk(top="/data2/wangxuedong/mhc_test_data/first_review/race_average_bin_1000/"):
    for file in files:   
        for category in allcategories:
            if file.startswith(category):
                filename=file[:file.index(".")]
                filepath=os.path.join(curDir,file)
                #print(filepath)
                compareWith0_01Pvalue(mean_arr=mean_dict_different_race[category],std_arr=std_dict_different_race[category],filename=filename,filepath=filepath)

In [11]:
def generatedfandcsv(columns,dic,filename):

    row_names=list(dic.keys())
    a=[]
    bs=[]

    for x in row_names:
        a.append(x)
        bs.append(dic[x])

    dictforDF=dict()
    dictforDF[columns[0]]=a
    dictforDF[columns[1]]=bs


    df_index=pd.DataFrame(dictforDF)
    df_index.to_csv("../race_start_length/"+filename+"_start_lenth.csv")
    return df_index


In [12]:
def checkContinuous1(arr):
    res=dict()
    i=0
    while i<len(arr)-1:
        if arr[i]==1:
            count=1
            for j in range(i+1,len(arr),1):
                if arr[j]==1:
                    count+=1                    
                else:                   
                    break
            res[i]=count
            i+=count
        else:
            i+=1
            continue
    return res
            


In [13]:
def getstartAndLength(filepath,filename):
    csv_matrix = np.loadtxt(open(filepath,"rb"),delimiter=",",skiprows=0)
    res=checkContinuous1(csv_matrix)
    df=generatedfandcsv(columns=["start","length"],dic=res,filename=filename)
    return df

In [14]:
for curDir, dirs, files in os.walk(top="../race_compare_with_01pvalue/"):
    for file in files:

        filename=file[:file.index(".")]
        filepath=os.path.join(curDir,file)
        getstartAndLength(filepath=filepath,filename=filename)

In [15]:

def countNumofLength(dic,filepath):
    csv_data = pd.read_csv(filepath, low_memory = False)
    csv_df = pd.DataFrame(csv_data)
    arr=np.array(csv_df['length'])
    #print(dic)
    for i in range(len(arr)):
        #print("before iteration",dic)
        if arr[i] in dic:
            #print("in iteration not",dic)
            dic[arr[i]]+=1
        else:
            dic[arr[i]]=1
            #print("in iteration",dic)
    #print(dic)
    dic = {k: v for k, v in sorted(dic.items())}

    return dic

In [16]:

race_dic_except_mhc=dict()
for category in allcategories:
    race_dic_except_mhc[category]=dict()

for curDir, dirs, files in os.walk(top="../race_start_length/"):
    for category in allcategories:
        mydic=dict()
        for file in files:        
            if file.startswith(category):
                filename=file[:file.index(".")]
                if "mhc" in filename:
                    continue          
                else:                  
                    filepath=os.path.join(curDir,file)
                    mydic=countNumofLength(mydic,filepath=filepath)
        race_dic_except_mhc[category]=mydic

In [17]:
race_dic_except_mhc#{race:{length of continuous '1': count}} for split_chr...

{'ACB': {1: 2053, 2: 413, 3: 99, 4: 26, 5: 10, 6: 3, 7: 1, 8: 2, 10: 1},
 'ASW': {1: 2129, 2: 403, 3: 102, 4: 31, 5: 6, 6: 2, 7: 1, 8: 1},
 'BEB': {1: 2032, 2: 448, 3: 105, 4: 29, 5: 10, 6: 2, 7: 3, 8: 1},
 'CDX': {1: 2049, 2: 437, 3: 104, 4: 33, 5: 12, 6: 3, 7: 1, 8: 1, 10: 1},
 'CEU': {1: 2109, 2: 432, 3: 102, 4: 28, 5: 12, 6: 4, 7: 4, 13: 1},
 'CHB': {1: 1991, 2: 418, 3: 104, 4: 21, 5: 14, 6: 8, 7: 6},
 'CHS': {1: 2037, 2: 457, 3: 93, 4: 30, 5: 11, 6: 4, 7: 3},
 'CLM': {1: 2040, 2: 412, 3: 108, 4: 31, 5: 6, 6: 5, 7: 1, 8: 2, 9: 1, 13: 1},
 'ESN': {1: 2046, 2: 423, 3: 111, 4: 27, 5: 3, 6: 3, 7: 5, 8: 2},
 'FIN': {1: 2129, 2: 436, 3: 116, 4: 30, 5: 15, 6: 6, 7: 3},
 'GBR': {1: 2069, 2: 439, 3: 97, 4: 33, 5: 15, 6: 2, 7: 2, 8: 2, 13: 1},
 'GIH': {1: 2001, 2: 448, 3: 95, 4: 31, 5: 13, 6: 3, 7: 3, 9: 1, 10: 1},
 'GWD': {1: 2082, 2: 417, 3: 99, 4: 31, 5: 5, 6: 3, 7: 2, 8: 2, 9: 1},
 'IBS': {1: 2033, 2: 415, 3: 98, 4: 29, 5: 6, 6: 3, 7: 4, 8: 1, 13: 1},
 'ITU': {1: 2024, 2: 445, 3: 106, 4:

In [18]:
mhc_final_dict=dict()
for curDir, dirs, files in os.walk(top="../race_start_length/"):
    for category in allcategories:
        mhcdic=dict()
        for file in files:   
            if file.startswith(category):
                filename=file[:file.index(".")]
                if "mhc" in filename:
                    filepath=os.path.join(curDir,file)
                    mhcdic=countNumofLength(mhcdic,filepath=filepath)
        mhc_final_dict[category]=mhcdic
                  

In [19]:
mhc_final_dict#{race:{length of continuous '1': count}} for mhc

{'ACB': {1: 2, 3: 1, 5: 1, 15: 1, 21: 1, 28: 1},
 'ASW': {1: 2, 2: 1, 3: 1, 4: 1, 18: 1, 19: 1, 28: 1},
 'BEB': {2: 2, 3: 1, 4: 1, 5: 1, 15: 1, 21: 1, 28: 1},
 'CDX': {1: 3, 2: 3, 4: 1, 5: 2, 21: 1, 29: 1},
 'CEU': {2: 2, 3: 3, 5: 1, 7: 1, 21: 1, 28: 1},
 'CHB': {1: 1, 3: 1, 5: 2, 9: 2, 21: 1, 28: 1},
 'CHS': {1: 2, 3: 1, 4: 2, 5: 1, 8: 1, 21: 1, 23: 1},
 'CLM': {1: 2, 2: 2, 3: 1, 5: 2, 9: 1, 21: 1, 28: 1},
 'ESN': {1: 2, 3: 1, 4: 1, 5: 1, 18: 1, 21: 1, 22: 1},
 'FIN': {1: 2, 3: 3, 4: 1, 5: 2, 21: 1, 28: 1},
 'GBR': {1: 1, 2: 3, 3: 4, 5: 1, 21: 1, 28: 1},
 'GIH': {1: 2, 2: 2, 3: 1, 5: 2, 9: 1, 21: 1, 28: 1},
 'GWD': {1: 2, 3: 1, 5: 1, 18: 1, 19: 1, 28: 1},
 'IBS': {1: 1, 2: 2, 3: 1, 4: 1, 6: 1, 8: 1, 21: 1, 28: 1},
 'ITU': {1: 1, 2: 3, 3: 1, 5: 1, 15: 1, 19: 1, 28: 1},
 'JPT': {1: 2, 3: 1, 5: 1, 8: 1, 9: 1, 21: 1, 28: 1},
 'KHV': {1: 1, 2: 1, 3: 1, 5: 1, 18: 1, 21: 1, 29: 1},
 'LWK': {1: 2, 2: 1, 3: 1, 4: 1, 5: 1, 15: 1, 19: 1, 24: 1},
 'MSL': {1: 2, 3: 1, 4: 1, 15: 1, 22: 1, 28: 1},
 

In [20]:
arr1=[]
for _,v in race_dic_except_mhc.items():
    for k2,arr in v.items():
        for i in range(arr):
            arr1.append(k2)
for _,v in mhc_final_dict.items():
    for k2,arr in v.items():
        for i in range(arr):
            arr1.append(k2)



In [21]:
mean_1=np.mean(arr1)
dev1=np.std(arr1)
print("mean",mean_1)
print("dev",dev1)
mid_res=stats.norm.ppf(0.9999999999,mean_1,dev1)
mid_res

mean 1.330768670309654
dev 0.9799511675327854


7.564572102242691

In [22]:
def getAllPositions(filepath):
    pos=[]
    df=pd.read_csv(filepath)
    pos=df["Unnamed: 0"].to_list()    
    return pos

In [23]:
def findPosition(filepath,start,length):
    arr=[]
    allpos=getAllPositions(filepath=filepath)
    arr.append(allpos[start*1000])
    if (start+length)*1000-1<len(allpos):
        end=allpos[(start+length)*1000-1]
    else:
        end=allpos[-1]
    arr.append(end)
    return arr



In [27]:
#for split_chr..., get complementary region
#/data2/wangxuedong/mhc_test_data/first_review/race_start_length/ACB_chr3_start_lenth.csv
def GetPosition(filepath_start_length,threshold,res):

    csv_data = pd.read_csv(filepath_start_length, low_memory = False)
    csv_df = pd.DataFrame(csv_data)
    arr_start=np.array(csv_df['start'])
    arr_length=np.array(csv_df['length'])
    mydic=dict()
    midarr=[]
    
    for i in range(len(arr_start)):
        mydic[arr_start[i]]=arr_length[i]
    
    name=filepath_start_length.split("/")[-1].split(".")[0].split("_")[1]
    name1=filepath_start_length.split("/")[-1].split(".")[0].split("_")[2]
    if name=="chr6":

        if name1!="mhcpart":
            filepath="/data2/wangxuedong/mhc_test_data/first_review/race_integration/"+name+"_"+name1+".csv"
    else:
        filepath="/data2/wangxuedong/mhc_test_data/first_review/race_integration/"+name+".csv"

    # filename_components=(filepath.split("/")[-1]).split("_start_lenth.csv")[0].split("_")
    # findfoldername="split_"+filename_components[2]
    # findfoldername_filename=filename_components[3]
    # filepath="../"+findfoldername+"/"+findfoldername_filename+".vcf.gz"

    for k,v in mydic.items():
        if v>=threshold:
            print("filepath",filepath)
            midarr.append(findPosition(filepath=filepath,start=k,length=v))
            
        else:
            continue
        
    if midarr!=[]:
        if name=="chr6":
            if name1!="mhcpart":
                res[name+"_"+name1]=midarr
        else:
            res[name]=midarr

    # if midarr!=[]:
    #     res[findfoldername+"/"+findfoldername_filename]=midarr
    return res

In [28]:
all_race_positons=dict()
for category in allcategories:
    all_race_positons[category]=dict()

for curDir, dirs, files in os.walk(top="../race_start_length/"):        
    for category in allcategories:
        res=dict()
        for file in files:
            filename=file[:file.index(".")]
            if "mhc" in filename:
                continue
            else:
                filerace=filename.split("_")[0]
                if filerace==category:                   
                    filepath=os.path.join(curDir,file)
                    res=GetPosition(filepath_start_length=filepath,threshold=8,res=res)
                    all_race_positons[category]=res


filepath /data2/wangxuedong/mhc_test_data/first_review/race_integration/chr3.csv


filepath /data2/wangxuedong/mhc_test_data/first_review/race_integration/chr4.csv
filepath /data2/wangxuedong/mhc_test_data/first_review/race_integration/chr5.csv
filepath /data2/wangxuedong/mhc_test_data/first_review/race_integration/chr1.csv
filepath /data2/wangxuedong/mhc_test_data/first_review/race_integration/chr13.csv
filepath /data2/wangxuedong/mhc_test_data/first_review/race_integration/chr7.csv
filepath /data2/wangxuedong/mhc_test_data/first_review/race_integration/chr15.csv
filepath /data2/wangxuedong/mhc_test_data/first_review/race_integration/chr17.csv
filepath /data2/wangxuedong/mhc_test_data/first_review/race_integration/chr3.csv
filepath /data2/wangxuedong/mhc_test_data/first_review/race_integration/chr3.csv
filepath /data2/wangxuedong/mhc_test_data/first_review/race_integration/chr3.csv
filepath /data2/wangxuedong/mhc_test_data/first_review/race_integration/chr17.csv
filepath /data2/wangxuedong/mhc_test_data/first_review/race_integration/chr1.csv
filepath /data2/wangxued

In [29]:
all_race_positons

{'ACB': {'chr3': [[98029388, 98331063]],
  'chr4': [[8996392, 9139446]],
  'chr5': [[34277945, 34574089]]},
 'ASW': {'chr1': [[248297800, 248584618]]},
 'BEB': {'chr13': [[55108214, 55447878]]},
 'CDX': {'chr7': [[64033450, 64433853]], 'chr15': [[24061424, 24264850]]},
 'CEU': {'chr17': [[45825257, 46352295]]},
 'CHB': {},
 'CHS': {},
 'CLM': {'chr3': [[95042564, 95345546],
   [98029388, 98261301],
   [163846034, 164208695]],
  'chr17': [[45825257, 46352295]]},
 'ESN': {'chr1': [[248297800, 248584618]], 'chr5': [[34277945, 34574089]]},
 'FIN': {},
 'GBR': {'chr1': [[227003796, 227338558]],
  'chr3': [[98029388, 98261301]],
  'chr17': [[45825257, 46352295]]},
 'GIH': {'chr3': [[98029388, 98331063]], 'chr14': [[19806333, 20078222]]},
 'GWD': {'chr1': [[248297800, 248584618]],
  'chr5': [[34277945, 34574089]],
  'chr7': [[11926418, 12230106]]},
 'IBS': {'chr1': [[227003796, 227338558]], 'chr17': [[45825257, 46352295]]},
 'ITU': {'chr17': [[52570659, 52892893]]},
 'JPT': {},
 'KHV': {},
 '

In [30]:
#for mhc get complementary region
def GetPositionformhc(filepath_start_length,threshold,res):

    csv_data = pd.read_csv(filepath_start_length, low_memory = False)
    csv_df = pd.DataFrame(csv_data)
    arr_start=np.array(csv_df['start'])
    arr_length=np.array(csv_df['length'])
    mydic=dict()
    midarr=[]
    
    for i in range(len(arr_start)):
        mydic[arr_start[i]]=arr_length[i]

    filepath="/data2/wangxuedong/mhc_test_data/first_review/race_integration/chr6_mhcpart.csv"

    for k,v in mydic.items():
        if v>=threshold:
            print("filepath",filepath)
            midarr.append(findPosition(filepath=filepath,start=k,length=v))
            
        else:
            continue

    if midarr!=[]:
        res["mhc"]=midarr
    return res

In [31]:
mhc_race_positons=dict()
for category in allcategories:
    mhc_race_positons[category]=dict()

for curDir, dirs, files in os.walk(top="../race_start_length/"):        
    for category in allcategories:
        res=dict()
        for file in files:
            filename=file[:file.index(".")]
            if "mhc" in filename:
                filerace=filename.split("_")[0]
                if filerace==category:                   
                    filepath=os.path.join(curDir,file)
                    print(filepath)
                    res=GetPositionformhc(filepath_start_length=filepath,threshold=8,res=res)
                    mhc_race_positons[category]=res
                else:
                    continue
            else:
                continue


../race_start_length/ACB_chr6_mhcpart_start_lenth.csv
filepath /data2/wangxuedong/mhc_test_data/first_review/race_integration/chr6_mhcpart.csv
filepath /data2/wangxuedong/mhc_test_data/first_review/race_integration/chr6_mhcpart.csv
filepath /data2/wangxuedong/mhc_test_data/first_review/race_integration/chr6_mhcpart.csv
../race_start_length/ASW_chr6_mhcpart_start_lenth.csv
filepath /data2/wangxuedong/mhc_test_data/first_review/race_integration/chr6_mhcpart.csv
filepath /data2/wangxuedong/mhc_test_data/first_review/race_integration/chr6_mhcpart.csv
filepath /data2/wangxuedong/mhc_test_data/first_review/race_integration/chr6_mhcpart.csv
../race_start_length/BEB_chr6_mhcpart_start_lenth.csv
filepath /data2/wangxuedong/mhc_test_data/first_review/race_integration/chr6_mhcpart.csv
filepath /data2/wangxuedong/mhc_test_data/first_review/race_integration/chr6_mhcpart.csv
filepath /data2/wangxuedong/mhc_test_data/first_review/race_integration/chr6_mhcpart.csv
../race_start_length/CDX_chr6_mhcpart

In [32]:
mhc_race_positons

{'ACB': {'mhc': [[29720403, 30011739],
   [30994370, 31528792],
   [32212726, 32882258]]},
 'ASW': {'mhc': [[29720403, 30120966],
   [31052133, 31528792],
   [32212726, 32882258]]},
 'BEB': {'mhc': [[29720403, 30011739],
   [30994370, 31528792],
   [32212726, 32882258]]},
 'CDX': {'mhc': [[30994370, 31528792], [32212726, 32923168]]},
 'CEU': {'mhc': [[30994370, 31528792], [32212726, 32882258]]},
 'CHB': {'mhc': [[29720403, 29913914],
   [29939668, 30154438],
   [30994370, 31528792],
   [32212726, 32882258]]},
 'CHS': {'mhc': [[29720403, 29896285],
   [30994370, 31528792],
   [32397207, 32882258]]},
 'CLM': {'mhc': [[29720403, 29913914],
   [30994370, 31528792],
   [32212726, 32882258]]},
 'ESN': {'mhc': [[29720403, 30120966],
   [30994370, 31528792],
   [32423532, 32882258]]},
 'FIN': {'mhc': [[30994370, 31528792], [32212726, 32882258]]},
 'GBR': {'mhc': [[30994370, 31528792], [32212726, 32882258]]},
 'GIH': {'mhc': [[29720403, 29913914],
   [30994370, 31528792],
   [32212726, 32882258